In [ ]:
import requests
import pandas as pd
import pymssql
from datetime import datetime
import requests
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:


url = "https://api.linnworks.net/api/Auth/AuthorizeByApplication"

payload = {
    "ApplicationId": "ApplicationId",
    "ApplicationSecret": "pplicationSecret",
    "Token": "Token"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

# Parse JSON response
data = response.json()

# Assuming the token is inside a key named 'Token' or 'AccessToken' or similar:
access_token = data.get("Token")

print(access_token)


38837d53-801d-48b5-b025-841b2b90bbf4


In [ ]:
# --- Config ---
base_url = "https://eu-ext.linnworks.net/api/PurchaseOrder"
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": access_token  
}

# --- Step 1: Search purchase orders ---
search_url = f"{base_url}/Search_PurchaseOrders2"
payload = {
    "searchParameters": {
        "DateFrom": "2000-01-01T00:00:00",
        "DateTo": datetime.now().isoformat()
    },
    "entriesPerPage": 100,
    "pageNumber": 1
}

all_orders = []
while True:
    response = requests.post(search_url, json=payload, headers=headers)
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break

    data = response.json()
    orders = data.get("Result", [])
    all_orders.extend(orders)

    total_pages = data.get("TotalPages", 1)
    current_page = data.get("CurrentPageNumber", 1)
    print(f"Fetched page {current_page}/{total_pages}, {len(orders)} orders")

    if current_page >= total_pages:
        break

    payload["pageNumber"] += 1


# --- Step 2: Fetch purchase order details in parallel ---
details_url = f"{base_url}/Get_PurchaseOrder"

def fetch_details(order):
    pkPurchaseId = order["pkPurchaseID"]
    resp = requests.post(details_url, json={"pkPurchaseId": pkPurchaseId}, headers=headers)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"Failed {pkPurchaseId}: {resp.text}")
        return None

detailed_orders = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_details, order) for order in all_orders]
    for f in as_completed(futures):
        r = f.result()
        if r:
            detailed_orders.append(r)


# --- Step 3: Flatten PurchaseOrderItem (one row per item) ---
items_df = pd.json_normalize(
    detailed_orders,
    record_path=["PurchaseOrderItem"],
    meta=[
        ["PurchaseOrderHeader", "pkPurchaseID"],
        ["PurchaseOrderHeader", "ExternalInvoiceNumber"],
        ["PurchaseOrderHeader", "Status"],
        ["PurchaseOrderHeader", "DateOfPurchase"],
        ["PurchaseOrderHeader", "DateOfDelivery"],
        ["PurchaseOrderHeader", "TotalCost"]
    ],
    errors="ignore"
)

# Prefix columns for clarity
items_df = items_df.rename(
    columns=lambda x: f"PurchaseOrderItem.{x}" if x not in [
        "PurchaseOrderHeader.pkPurchaseID",
        "PurchaseOrderHeader.ExternalInvoiceNumber",
        "PurchaseOrderHeader.Status",
        "PurchaseOrderHeader.DateOfPurchase",
        "PurchaseOrderHeader.DateOfDelivery",
        "PurchaseOrderHeader.TotalCost"
    ] else x
)

# --- Step 4: Flatten DeliveredRecords ---
deliveries_df = pd.json_normalize(
    detailed_orders,
    record_path=["DeliveredRecords"],
    meta=[["PurchaseOrderHeader", "pkPurchaseID"]],
    errors="ignore"
)

# Prefix delivery columns
deliveries_df = deliveries_df.rename(
    columns=lambda x: f"DeliveredRecords.{x}" if x not in ["PurchaseOrderHeader.pkPurchaseID"] else x
)

# --- Step 5: Merge delivered records into items ---
df = items_df.merge(
    deliveries_df,
    left_on=["PurchaseOrderItem.pkPurchaseItemId", "PurchaseOrderHeader.pkPurchaseID"],
    right_on=["DeliveredRecords.fkPurchaseItemId", "PurchaseOrderHeader.pkPurchaseID"],
    how="left"
)

pd.set_option("display.max_columns", None)



Fetched page 1/4, 100 orders
Fetched page 2/4, 100 orders
Fetched page 3/4, 100 orders
Fetched page 4/4, 57 orders
Failed 4b60f904-78e4-469e-bc71-8623858df993: "API calls quota exceeded! Maximum admitted 250 per Minute."
Failed 621c8bc3-172c-42bc-adc3-6c727bfe51a1: "API calls quota exceeded! Maximum admitted 250 per Minute."
Failed accacd02-86c6-428d-b91e-7ffe999f6866: "API calls quota exceeded! Maximum admitted 250 per Minute."
Failed a68ed207-dfb3-4fab-b701-45348a0006fd: "API calls quota exceeded! Maximum admitted 250 per Minute."
Failed e4217603-6e99-4145-ab14-85c8dd40f767: "API calls quota exceeded! Maximum admitted 250 per Minute."
Failed 226aa7a4-054e-4d97-8b2f-baa606aabd08: "API calls quota exceeded! Maximum admitted 250 per Minute."
Failed 22034482-864c-4148-84c6-73b79f3b8596: "API calls quota exceeded! Maximum admitted 250 per Minute."
Failed 444a50bb-a25b-4773-bb4e-5246d60b8df1: "API calls quota exceeded! Maximum admitted 250 per Minute."
Failed 1c587ef3-1f9d-4e6b-91aa-1a818a

In [37]:
df = df.rename(columns={
    "PurchaseOrderItem.pkPurchaseItemId": "PurchaseOrderItem_pkPurchaseItemId",
    "PurchaseOrderItem.fkStockItemId": "PurchaseOrderItem_fkStockItemId",
    "PurchaseOrderItem.StockItemIntId": "PurchaseOrderItem_StockItemIntId",
    "PurchaseOrderItem.Quantity": "PurchaseOrderItem_Quantity",
    "PurchaseOrderItem.Cost": "PurchaseOrderItem_Cost",
    "PurchaseOrderItem.Delivered": "PurchaseOrderItem_Delivered",
    "PurchaseOrderItem.TaxRate": "PurchaseOrderItem_TaxRate",
    "PurchaseOrderItem.Tax": "PurchaseOrderItem_Tax",
    "PurchaseOrderItem.PackQuantity": "PurchaseOrderItem_PackQuantity",
    "PurchaseOrderItem.PackSize": "PurchaseOrderItem_PackSize",
    "PurchaseOrderItem.SKU": "PurchaseOrderItem_SKU",
    "PurchaseOrderItem.ItemTitle": "PurchaseOrderItem_ItemTitle",
    "PurchaseOrderItem.InventoryTrackingType": "PurchaseOrderItem_InventoryTrackingType",
    "PurchaseOrderItem.IsDeleted": "PurchaseOrderItem_IsDeleted",
    "PurchaseOrderItem.SortOrder": "PurchaseOrderItem_SortOrder",
    "PurchaseOrderItem.DimHeight": "PurchaseOrderItem_DimHeight",
    "PurchaseOrderItem.DimWidth": "PurchaseOrderItem_DimWidth",
    "PurchaseOrderItem.BarcodeNumber": "PurchaseOrderItem_BarcodeNumber",
    "PurchaseOrderItem.DimDepth": "PurchaseOrderItem_DimDepth",
    "PurchaseOrderItem.BoundToOpenOrdersItems": "PurchaseOrderItem_BoundToOpenOrdersItems",
    "PurchaseOrderItem.QuantityBoundToOpenOrdersItems": "PurchaseOrderItem_QuantityBoundToOpenOrdersItems",
    "PurchaseOrderItem.SupplierCode": "PurchaseOrderItem_SupplierCode",
    "PurchaseOrderItem.SupplierBarcode": "PurchaseOrderItem_SupplierBarcode",
    "PurchaseOrderItem.SkuGroupIds": "PurchaseOrderItem_SkuGroupIds",
    "PurchaseOrderItem.BinRack": "PurchaseOrderItem_BinRack",
    "PurchaseOrderHeader.pkPurchaseID": "PurchaseOrderHeader_pkPurchaseID",
    "PurchaseOrderHeader.ExternalInvoiceNumber": "PurchaseOrderHeader_ExternalInvoiceNumber",
    "PurchaseOrderHeader.Status": "PurchaseOrderHeader_Status",
    "PurchaseOrderHeader.DateOfPurchase": "PurchaseOrderHeader_DateOfPurchase",
    "PurchaseOrderHeader.DateOfDelivery": "PurchaseOrderHeader_DateOfDelivery",
    "PurchaseOrderHeader.TotalCost": "PurchaseOrderHeader_TotalCost",
    "DeliveredRecords.pkDeliveryRecordId": "DeliveredRecords_pkDeliveryRecordId",
    "DeliveredRecords.fkPurchaseItemId": "DeliveredRecords_fkPurchaseItemId",
    "DeliveredRecords.fkStockLocationId": "DeliveredRecords_fkStockLocationId",
    "DeliveredRecords.UnitCost": "DeliveredRecords_UnitCost",
    "DeliveredRecords.DeliveredQuantity": "DeliveredRecords_DeliveredQuantity",
    "DeliveredRecords.CreatedDateTime": "DeliveredRecords_CreatedDateTime",
    "DeliveredRecords.fkBatchInventoryId": "DeliveredRecords_fkBatchInventoryId",
    "DeliveredRecords.ModifiedDateTime": "DeliveredRecords_ModifiedDateTime",
})


In [ ]:



# Replace NaN with None for SQL
df = df.where(pd.notnull(df), None)

df = df.dropna(how='all')


datetime_cols = [col for col in df.columns if 'Date' in col or 'date' in col]
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')



table_name = "lw.FullPurchaseOrders"
batch_size = 1000

columns = df.columns.tolist()
column_str = ", ".join(f"[{c}]" for c in columns)  # wrap in brackets to be safe
placeholders = ", ".join(["%s"] * len(columns))


for start in range(0, len(df), batch_size):
    batch = df.iloc[start:start+batch_size]
    if batch.empty:
        continue
    values = [tuple(x) for x in batch.to_numpy() if any(x)]  # skip fully empty rows
    if not values:
        continue
    query = f"INSERT INTO {table_name} ({column_str}) VALUES ({placeholders})"
    cursor.executemany(query, values)
    conn.commit()
    print(f"Inserted rows {start} to {start + len(values)}")

conn.close()
print("Upload complete!")


ValueError: NaTType does not support strftime

In [55]:
df.count()

PurchaseOrderItem.pkPurchaseItemId                  4397
PurchaseOrderItem.fkStockItemId                     4397
PurchaseOrderItem.StockItemIntId                    4397
PurchaseOrderItem.Quantity                          4397
PurchaseOrderItem.Cost                              4397
PurchaseOrderItem.Delivered                         4397
PurchaseOrderItem.TaxRate                           4397
PurchaseOrderItem.Tax                               4397
PurchaseOrderItem.PackQuantity                      4397
PurchaseOrderItem.PackSize                          4397
PurchaseOrderItem.SKU                               4397
PurchaseOrderItem.ItemTitle                         4397
PurchaseOrderItem.InventoryTrackingType             4397
PurchaseOrderItem.IsDeleted                         4397
PurchaseOrderItem.SortOrder                         4397
PurchaseOrderItem.DimHeight                         4397
PurchaseOrderItem.DimWidth                          4397
PurchaseOrderItem.BarcodeNumber